In [6]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta

# Set up project directory
project_dir = 'data_csv'
data_dir = os.path.join(project_dir, '')
os.makedirs(data_dir, exist_ok=True)

def calculate_moving_average(data, window):
    """
    이동 평균을 계산하는 함수. 데이터가 부족한 경우 현재까지의 평균을 사용.
    
    Args:
        data (pd.Series): 주식 종가 데이터 시리즈.
        window (int): 이동 평균을 계산할 기간.
    
    Returns:
        pd.Series: 이동 평균 데이터 시리즈.
    """
    ma = data.rolling(window=window, min_periods=1).mean()
    return ma

def get_stock_data(ticker, start_date, end_date, interval='1d'):
    """
    주어진 주식 코드와 기간에 해당하는 주식 데이터를 받아오는 함수.

    Args:
        ticker (str): 주식 코드.
        start_date (str): 데이터의 시작 날짜 (YYYY-MM-DD 형식).
        end_date (str): 데이터의 종료 날짜 (YYYY-MM-DD 형식).
        interval (str): 데이터 간격 (1d, 1m 등).

    Returns:
        pd.DataFrame: 주식 데이터 프레임.
    """
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data = data[['Close', 'Volume']]  # 종가 및 거래량 데이터 사용
    data = data.reset_index()
    data.columns = ['Date', 'Close', 'Volume']
    
    # 이동평균선 데이터 계산
    ma_columns = {}
    for ma in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] + list(range(70, 710, 10)):
        ma_columns[f'SMA_{ma}'] = calculate_moving_average(data['Close'], ma)
        ma_columns[f'VMA_{ma}'] = calculate_moving_average(data['Volume'], ma)
    
    ma_df = pd.DataFrame(ma_columns)
    data = pd.concat([data, ma_df], axis=1)
    
    # NaN 값을 앞쪽 값으로 채우기
    data.ffill(inplace=True)
    data.bfill(inplace=True)  # 앞쪽에 값이 없을 경우 뒤쪽 값으로 채우기

    # 추가 데이터 계산
    data['365D_High'] = data['Close'].rolling(window=365, min_periods=1).max()
    data['365D_Low'] = data['Close'].rolling(window=365, min_periods=1).min()
    data['180D_High'] = data['Close'].rolling(window=180, min_periods=1).max()
    data['180D_Low'] = data['Close'].rolling(window=180, min_periods=1).min()
    data['90D_High'] = data['Close'].rolling(window=90, min_periods=1).max()
    data['90D_Low'] = data['Close'].rolling(window=90, min_periods=1).min()
    data['30D_High'] = data['Close'].rolling(window=30, min_periods=1).max()
    data['30D_Low'] = data['Close'].rolling(window=30, min_periods=1).min()
    data['AllTime_High'] = data['Close'].cummax()
    data['AllTime_Low'] = data['Close'].cummin()
    
    return data

def get_minute_stock_data(ticker, start_date, end_date):
    """
    1분봉 주식 데이터를 7일 단위로 받아오는 함수.

    Args:
        ticker (str): 주식 코드.
        start_date (str): 데이터의 시작 날짜 (YYYY-MM-DD 형식).
        end_date (str): 데이터의 종료 날짜 (YYYY-MM-DD 형식).

    Returns:
        pd.DataFrame: 주식 데이터 프레임.
    """
    all_data = []
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    delta = timedelta(days=7)

    while start < end:
        temp_end = min(start + delta, end)
        data = yf.download(ticker, start=start, end=temp_end, interval='1m')
        all_data.append(data)
        start = temp_end

    data = pd.concat(all_data)
    data = data[['Close', 'Volume']]  # 종가 및 거래량 데이터 사용
    data = data.reset_index()
    data.columns = ['Date', 'Close', 'Volume']
    
    # 이동평균선 데이터 계산
    ma_columns = {}
    for ma in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] + list(range(70, 710, 10)):
        ma_columns[f'SMA_{ma}'] = calculate_moving_average(data['Close'], ma)
        ma_columns[f'VMA_{ma}'] = calculate_moving_average(data['Volume'], ma)
    
    ma_df = pd.DataFrame(ma_columns)
    data = pd.concat([data, ma_df], axis=1)
    
    # NaN 값을 앞쪽 값으로 채우기
    data.fillna(method='ffill', inplace=True)
    data.fillna(method='bfill', inplace=True)  # 앞쪽에 값이 없을 경우 뒤쪽 값으로 채우기
    
    return data

def save_data_to_csv(data, filename):
    """
    주어진 데이터를 CSV 파일로 저장하는 함수.

    Args:
        data (pd.DataFrame): 저장할 데이터 프레임.
        filename (str): 저장할 CSV 파일의 이름.
    """
    data.to_csv(filename, index=False)
    print(f'Data saved to {filename}')

# Example usage
ticker = '000270.KS'
start_date = '2004-01-01'
end_date = '2023-01-01'
filename = os.path.join(data_dir, 'kia_stock_data.csv')

# 일봉 데이터 가져오기  
data = get_stock_data(ticker, start_date, end_date, interval='1d')
save_data_to_csv(data, filename)

# Example usage
ticker = '000270.KS'
start_date = '2022-01-02'
end_date = '2024-07-23'
filename = os.path.join(data_dir, 'kia_stock_testdata.csv')

# 일봉 데이터 가져오기
data = get_stock_data(ticker, start_date, end_date, interval='1d')
save_data_to_csv(data, filename)

# 1분봉 데이터 가져오기 (최대 7일 범위 내)
# minute_data = get_minute_stock_data(ticker, '2024-07-01', '2024-07-14')
# minute_filename = os.path.join(data_dir, 'kia_stock_minute_data.csv')
# save_data_to_csv(minute_data, minute_filename)


ModuleNotFoundError: No module named 'google.colab'